In [1]:
import pandas as pd
from tbdynamics.plotting import plot_model_vs_actual
import nevergrad as ng

# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.calib_utils import get_bcm, load_targets
from multiprocessing import cpu_count

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
pd.options.plotting.backend = "plotly"

In [3]:
params = {
    # "start_population_size": 2300000.0,
    # "seed_time": 1830.0,
    # "seed_num": 70.0,
    # "seed_duration": 20.0,
    # 'time_to_screening_end_asymp': 1.666
}
bcm = get_bcm(params)

In [4]:
# Set the number of workers for parallel optimization
orunner = optimize_model(bcm, opt_class=ng.optimizers.TwoPointsDE, num_workers=cpu_count())
for i in range(8):
    rec = orunner.minimize(1000)
mle_params = rec.value[1]
mle_params

{'contact_rate': 0.015235702440018003,
 'start_population_size': 2127095.819760846,
 'rr_infection_latent': 0.3683149419458062,
 'rr_infection_recovered': 0.44270788345353584,
 'progression_multiplier': 1.2078031584125082,
 'seed_time': 1807.8638432309192,
 'seed_num': 76.86277884429947,
 'seed_duration': 16.365794782204087,
 'smear_positive_death_rate': 0.4160065770952812,
 'smear_negative_death_rate': 0.029424178471320683,
 'smear_positive_self_recovery': 0.21678678185214179,
 'smear_negative_self_recovery': 0.18268089533131407,
 'screening_scaleup_shape': 0.11532502559402759,
 'screening_inflection_time': 1991.9639686143937,
 'time_to_screening_end_asymp': 1.2182257186335912,
 'detection_reduction': 0.32347397459677973,
 'contact_reduction': 0.16029227617201836}

In [5]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets()

In [6]:
plot_model_vs_actual(
    derived_df_0, targets['total_population'], "total_population", "Population", "Modelled vs Data"
)

In [7]:
plot_model_vs_actual(
    derived_df_0, targets['percentage_latent'], "percentage_latent", "Percentage latent", "Modelled vs Data"
)

In [8]:
plot_model_vs_actual(derived_df_0, targets['adults_prevalence_pulmonary'], 'adults_prevalence_pulmonary', 'Infectious prevalence', 'Modelled vs Estimation from 2017 prevalence survey')

In [9]:
plot_model_vs_actual(
    derived_df_0, targets['incidence'], "incidence", "Incidence", "Modelled vs Data"
)

In [10]:
plot_model_vs_actual(
    derived_df_0, targets['notification'], "notification", "Notifications", "Modelled vs Data"
)

In [11]:
plot_model_vs_actual(
    derived_df_0, targets['mortality_raw'], "mortality_raw", "Mortality raw", "Modelled vs Data"
)

In [12]:
# Plot each series. Each call will automatically plot on the same figure.
derived_df_0['prop_smear_positive'].plot()

In [13]:
derived_df_0['prop_smear_negative'].plot()

In [14]:
derived_df_0['prop_extrapulmonary'].plot()

In [15]:
derived_df_0['prevalence_smear_positive'].plot()

In [16]:
derived_df_0['case_detection_rate'].plot()

In [17]:
derived_df_0['detection_rate'].plot()

In [18]:
from tbdynamics.extended_gamma import ExtendedGammaPrior

In [19]:
median = 1.3
upper_ci = 1.4
extended_gamma_prior = ExtendedGammaPrior.from_median("example_prior", median, upper_ci)
print(f"Shape: {extended_gamma_prior.shape}, Scale: {extended_gamma_prior.scale}")

Shape: 682.5732944953737, Scale: 0.0019054878340583135
